In [56]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 20: Particle Swarm ---</h2><p>Suddenly, the GPU contacts you, asking for <span title="...as if millions of graphics pipelines suddenly cried out for help, but suddenly started working on something else instead because they all have to do the same thing at the same time and can't spend very long asking for help.">help</span>. Someone has asked it to simulate <em>too many particles</em>, and it won't be able to finish them all in time to render the next frame at this rate.</p>
<p>It transmits to you a buffer (your puzzle input) listing each particle in order (starting with particle <code>0</code>, then particle <code>1</code>, particle <code>2</code>, and so on). For each particle, it provides the <code>X</code>, <code>Y</code>, and <code>Z</code> coordinates for the particle's position (<code>p</code>), velocity (<code>v</code>), and acceleration (<code>a</code>), each in the format <code>&lt;X,Y,Z&gt;</code>.</p>
<p>Each tick, all particles are updated simultaneously. A particle's properties are updated in the following order:</p>
<ul>
<li>Increase the <code>X</code> velocity by the <code>X</code> acceleration.</li>
<li>Increase the <code>Y</code> velocity by the <code>Y</code> acceleration.</li>
<li>Increase the <code>Z</code> velocity by the <code>Z</code> acceleration.</li>
<li>Increase the <code>X</code> position by the <code>X</code> velocity.</li>
<li>Increase the <code>Y</code> position by the <code>Y</code> velocity.</li>
<li>Increase the <code>Z</code> position by the <code>Z</code> velocity.</li>
</ul>
<p>Because of seemingly tenuous rationale involving <a href="https://en.wikipedia.org/wiki/Z-buffering">z-buffering</a>, the GPU would like to know which particle will stay closest to position <code>&lt;0,0,0&gt;</code> in the long term. Measure this using the <a href="https://en.wikipedia.org/wiki/Taxicab_geometry">Manhattan distance</a>, which in this situation is simply the sum of the absolute values of a particle's <code>X</code>, <code>Y</code>, and <code>Z</code> position.</p>
<p>For example, suppose you are only given two particles, both of which stay entirely on the X-axis (for simplicity). Drawing the current states of particles <code>0</code> and <code>1</code> (in that order) with an adjacent a number line and diagram of current <code>X</code> positions (marked in parentheses), the following would take place:</p>
<pre><code>p=&lt; 3,0,0&gt;, v=&lt; 2,0,0&gt;, a=&lt;-1,0,0&gt;    -4 -3 -2 -1  0  1  2  3  4
p=&lt; 4,0,0&gt;, v=&lt; 0,0,0&gt;, a=&lt;-2,0,0&gt;                         (0)(1)

p=&lt; 4,0,0&gt;, v=&lt; 1,0,0&gt;, a=&lt;-1,0,0&gt; -4 -3 -2 -1 0 1 2 3 4
p=&lt; 2,0,0&gt;, v=&lt;-2,0,0&gt;, a=&lt;-2,0,0&gt; (1) (0)

p=&lt; 4,0,0&gt;, v=&lt; 0,0,0&gt;, a=&lt;-1,0,0&gt; -4 -3 -2 -1 0 1 2 3 4
p=&lt;-2,0,0&gt;, v=&lt;-4,0,0&gt;, a=&lt;-2,0,0&gt; (1) (0)

p=&lt; 3,0,0&gt;, v=&lt;-1,0,0&gt;, a=&lt;-1,0,0&gt; -4 -3 -2 -1 0 1 2 3 4
p=&lt;-8,0,0&gt;, v=&lt;-6,0,0&gt;, a=&lt;-2,0,0&gt; (0)  
</code></pre>

<p>At this point, particle <code>1</code> will never be closer to <code>&lt;0,0,0&gt;</code> than particle <code>0</code>, and so, in the long run, particle <code>0</code> will stay closest.</p>
<p><em>Which particle will stay closest to position <code>&lt;0,0,0&gt;</code></em> in the long term?</p>
</article>


In [57]:
example = """
p=< 3,0,0>, v=< 2,0,0>, a=<-1,0,0>   -4 -3 -2 -1  0  1  2  3  4
p=< 4,0,0>, v=< 0,0,0>, a=<-2,0,0>                        (0)(1)
p=< 4,0,0>, v=< 1,0,0>, a=<-1,0,0>   -4 -3 -2 -1  0  1  2  3  4
p=< 2,0,0>, v=<-2,0,0>, a=<-2,0,0>                     (1)   (0)
p=< 4,0,0>, v=< 0,0,0>, a=<-1,0,0>   -4 -3 -2 -1  0  1  2  3  4
p=<-2,0,0>, v=<-4,0,0>, a=<-2,0,0>         (1)               (0)
p=< 3,0,0>, v=<-1,0,0>, a=<-1,0,0>   -4 -3 -2 -1  0  1  2  3  4
p=<-8,0,0>, v=<-6,0,0>, a=<-2,0,0>                        (0)   
"""

example_particles = """
p=< 3,0,0>, v=< 2,0,0>, a=<-1,0,0>
p=< 4,0,0>, v=< 0,0,0>, a=<-2,0,0>
"""

In [63]:
from math import sqrt
from re import findall
from typing import Generator

from more_itertools import last
from numpy import roots


@dataclass(order=True)
class Point:
    x: int
    y: int
    z: int
    dx: int
    dy: int
    dz: int
    ddx: int
    ddy: int
    ddz: int

    def distance(self, other: Point) -> int:
        return abs(self.x - other.x) + abs(self.y - other.y) + abs(self.z - other.z)

    def manhatten_accelartion_from_origin(self) -> int:
        return abs(self.ddx) + abs(self.ddy) + abs(self.ddz)

    def collides_with(self, other: Point) -> set[float]:
        x = self.x - other.x
        dx = self.dx - other.dx
        ddx = (self.ddx - other.ddx) / 2
        dx += ddx

        y = self.y - other.y
        dy = self.dy - other.dy
        ddy = (self.ddy - other.ddy) / 2
        dy += ddy

        z = self.z - other.z
        dz = self.dz - other.dz
        ddz = (self.ddz - other.ddz) / 2
        dz += ddz

        if y == dy == ddy == z == dz == ddz == 0:
            return self.roots(ddx, dx, x)

        return self.roots(ddx, dx, x) & self.roots(ddy, dy, y) & self.roots(ddz, dz, z)

    @staticmethod
    def roots(a, b, c) -> set[float]:
        if a == 0 and b == 0:
            return {0} if c == 0 else set()

        if a == 0:
            return {-c / b}

        d = b * b - 4 * a * c
        if d < 0:
            return set()

        return {(-b - sqrt(d)) / (2 * a), (-b + sqrt(d)) / (2 * a)}


def parse(data: str) -> Generator[Point]:
    return (
        Point(*[int(i) for i in findall(r"-?\d+", l)])
        for l in data.strip().splitlines()
    )


def particle_will_stay_closest_to_origin(data: str) -> int:
    return last(
        min(
            (p.manhatten_accelartion_from_origin(), i)
            for i, p in enumerate(parse(data))
        )
    )


print(f"Example: {particle_will_stay_closest_to_origin(example_particles)}")

Example: 0


In [64]:
with open("../input/day20.txt") as f:
    puzzle = f.read()

print(f"Part I: {particle_will_stay_closest_to_origin(puzzle)}")

Part I: 376


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>376</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>To simplify the problem further, the GPU would like to remove any particles that <em>collide</em>. Particles collide if their positions ever <em>exactly match</em>. Because particles are updated simultaneously, <em>more than two particles</em> can collide at the same time and place.  Once particles collide, they are removed and cannot collide with anything else after that tick.</p>
<p>For example:</p>
<pre><code>p=&lt;-6,0,0&gt;, v=&lt; 3,0,0&gt;, a=&lt; 0,0,0&gt;    
p=&lt;-4,0,0&gt;, v=&lt; 2,0,0&gt;, a=&lt; 0,0,0&gt;    -6 -5 -4 -3 -2 -1  0  1  2  3
p=&lt;-2,0,0&gt;, v=&lt; 1,0,0&gt;, a=&lt; 0,0,0&gt;    (0)   (1)   (2)            (3)
p=&lt; 3,0,0&gt;, v=&lt;-1,0,0&gt;, a=&lt; 0,0,0&gt;

p=&lt;-3,0,0&gt;, v=&lt; 3,0,0&gt;, a=&lt; 0,0,0&gt;  
p=&lt;-2,0,0&gt;, v=&lt; 2,0,0&gt;, a=&lt; 0,0,0&gt; -6 -5 -4 -3 -2 -1 0 1 2 3
p=&lt;-1,0,0&gt;, v=&lt; 1,0,0&gt;, a=&lt; 0,0,0&gt; (0)(1)(2) (3)  
p=&lt; 2,0,0&gt;, v=&lt;-1,0,0&gt;, a=&lt; 0,0,0&gt;

p=&lt; 0,0,0&gt;, v=&lt; 3,0,0&gt;, a=&lt; 0,0,0&gt;  
p=&lt; 0,0,0&gt;, v=&lt; 2,0,0&gt;, a=&lt; 0,0,0&gt; -6 -5 -4 -3 -2 -1 0 1 2 3
p=&lt; 0,0,0&gt;, v=&lt; 1,0,0&gt;, a=&lt; 0,0,0&gt; X (3)  
p=&lt; 1,0,0&gt;, v=&lt;-1,0,0&gt;, a=&lt; 0,0,0&gt;

------destroyed by collision------  
------destroyed by collision------ -6 -5 -4 -3 -2 -1 0 1 2 3
------destroyed by collision------ (3)  
p=&lt; 0,0,0&gt;, v=&lt;-1,0,0&gt;, a=&lt; 0,0,0&gt;
</code></pre>

<p>In this example, particles <code>0</code>, <code>1</code>, and <code>2</code> are simultaneously destroyed at the time and place marked <code>X</code>. On the next tick, particle <code>3</code> passes through unharmed.</p>
<p><em>How many particles are left</em> after all collisions are resolved?</p>
</article>

</main>


In [65]:
example = """
p=<-6,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=<-4,0,0>, v=< 2,0,0>, a=< 0,0,0>
p=<-2,0,0>, v=< 1,0,0>, a=< 0,0,0>
p=< 3,0,0>, v=<-1,0,0>, a=< 0,0,0>
"""


def count_noncolliders(data: str) -> int:
    particles = list(parse(data))
    n = len(particles)
    intersections = defaultdict(list)
    for i in range(n):
        for j in range(i + 1, n):
            for t in particles[i].collides_with(particles[j]):
                if t >= 0:
                    intersections[t].append((i, j))

    seen_at = {}
    for t, ints in sorted(intersections.items()):
        for i, j in ints:
            if i not in seen_at and j not in seen_at:
                seen_at[i] = t
                seen_at[j] = t
            elif i not in seen_at and abs(seen_at.get(j, inf) - t) < 0.001:
                seen_at[i] = t
            elif j not in seen_at and abs(seen_at.get(i, inf) - t) < 0.001:
                seen_at[j] = t

    return len(set(range(n)) - set(seen_at.keys()))


print(f"Example: {count_noncolliders(example)}")
print(f"Part II: {count_noncolliders(puzzle)}")

Example: 1
Part II: 574


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>574</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
